In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm.notebook import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [9]:
''' 파일 불러오기 '''
df = pd.read_csv("./산림복지/2019산림복지_본문_노이즈 제거.txt", sep='\t', encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [10]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [4]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

  0%|          | 0/6127 [00:00<?, ?it/s]

,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/waura21/221755822694,[코리아투데이뉴스=상주시] 상주시 2020년 상반기 정기인사...,2019.12.31.,[코리아투데이뉴스=상주시] 상주시 2020년 상반기 정기인사 이동조서-(2020. ...,0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,[(코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 ...
1,https://blog.naver.com/skfzhddlwkd/221755736686,[지방분권 대학생 서포터즈] 공공기관 지방이전과 혁신도시,2019.12.31.,안녕하세요~ 메리썽입니다.\n혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는...,1,안녕하세요 메리썽입니다 혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는데요 ...,"[(안녕하세요 메리썽입니다, 0, 12, None, []), (혁신도시에 대해서는 ..."
2,https://blog.naver.com/vipgim1390/221755564191,[인사] 경북 구미시,2019.12.31.,◇4급 승진 △경제기획국장 박수원 △문화체육관광국장 김회식 △의회사무국장 이근도 △...,2,4급 승진 경제기획국장 박수원 문화체육관광국장 김회식 의회사무국장 이근도 평생교육원...,[(4급 승진 경제기획국장 박수원 문화체육관광국장 김회식 의회사무국장 이근도 평생교...
3,https://blog.naver.com/aromaplayer/221755469475,경기창조경제혁신센터_로컬크리에이터_사후간담회_역시...,2019.12.31.,안녕하세요!\n반고 유은기(유진선)입니다.\n오늘은 2019년의 마지막날!! 입니다...,3,안녕하세요 반고 유은기 유진선 입니다 오늘은 2019년의 마지막날 입니다 올해는 정...,"[(안녕하세요 반고 유은기 유진선 입니다, 0, 20, None, []), (오늘은..."
4,https://blog.naver.com/sejongicon/221755302232,국립칠곡숲체원 사인/ 간판디자인,2019.12.31.,한국산림복지진흥원 국립칠곡숲체원 사인/ 간판 디자인 및 시공\nhttps://blo...,4,한국산림복지진흥원 국립칠곡숲체원 사인 간판 디자인 및 시공 https blog na...,[(한국산림복지진흥원 국립칠곡숲체원 사인 간판 디자인 및 시공 https blog ...
...,...,...,...,...,...,...,...
6122,https://blog.naver.com/vipgim1390/221431934615,[인사] 경북 김천시,2019.01.01.,◆4급 승진 △금오공대 파견 김용수 △경제산업국장 구영훈 △의회사무국장 안민 △행정...,6122,4급 승진 금오공대 파견 김용수 경제산업국장 구영훈 의회사무국장 안민 행정복지국장 ...,[(4급 승진 금오공대 파견 김용수 경제산업국장 구영훈 의회사무국장 안민 행정복지국...
6123,https://blog.naver.com/dltmf1427/221431893814,"2018년 마무리, 2019년 새해 목표는 세우셨나요? (부록...",2019.01.01.,안녕하세요.\n라포르커뮤니케이션 원쌤이에요.\n2019년 한 해 잘 시작하셨나요^^...,6123,안녕하세요 라포르커뮤니케이션 원쌤이에요 2019년 한 해 잘 시작하셨나요 나이 먹기...,"[(안녕하세요 라포르커뮤니케이션 원쌤이에요, 0, 21, None, []), (20..."
6124,https://blog.naver.com/rich2s/221431887654,2018년 12월 31일 동·서·남해안 및 내륙권 발전 특별법...,2019.01.01.,블로그\n부동산뉴스 95개의 글\n부동산뉴스\n목록열기\n2018년 12월 31일 ...,6124,블로그 부동산뉴스 95개의 글 부동산뉴스 목록열기 2018년 12월 31일 동 서 ...,[(블로그 부동산뉴스 95개의 글 부동산뉴스 목록열기 2018년 12월 31일 동 ...
6125,https://blog.naver.com/changsantree/221431590702,1월 1일 화요일,2019.01.01.,blog\n공지 목록\n공지글\n글 제목 작성일\n(1)\n공지 3월 27일 수요일...,6125,blog 공지 목록 공지글 글 제목 작성일 1 공지 3월 27일 수요일 2019 3...,[(blog 공지 목록 공지글 글 제목 작성일 1 공지 3월 27일 수요일 2019...


In [ ]:
# 데이터프레임을 100개 단위로 나누기
chunk_size = 100
num_chunks = len(df) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df[start_idx:end_idx]
    dfs[f"df_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

In [ ]:
# df = pd.read_csv("2019산림복지_세이브포인트.csv", encoding='UTF-8')
# df

In [ ]:
df["split_list"][0]

In [7]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("./산림복지/2019산림복지_본문_전처리.csv", index=False, encoding='UTF-8')

In [81]:
df2 = pd.read_csv("./산림복지/2019산림복지_본문_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx
0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,0
1,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤원해 평생학습원 이세인 ...,0
2,안녕하세요 메리썽입니다,1
3,혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는데요,1
4,현재 혁신도시 시즌2가 지속적으로 진행되고 있는 만큼 지금 시기에 이에 대해 이야기...,1
...,...,...
164724,의 건립비로 지출하는 기부금 9 중소기업협동조합법 에 따른 중소기업중앙회에 중소기업...,6126
164725,이하 이 호에서 같다 가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경...,6126
164726,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,6126
164727,한국국제학교 22 천진한국국제학교 23 테헤란한국학교 ㅇ 법정기부금단체 중 전문모금...,6126


In [16]:
dfs['df_1']['split_list'].index

RangeIndex(start=0, stop=100, step=1)

In [78]:
a = 62
for i in tqdm(dfs['df_%s'%a]['split_list'].index):
    for j in range(len(dfs['df_%s'%a]['split_list'][i])):
        split_str= {'split_str': dfs['df_%s'%a]['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

,split_str,org_idx
0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,0
1,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤원해 평생학습원 이세인 ...,0
2,안녕하세요 메리썽입니다,1
3,혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는데요,1
4,현재 혁신도시 시즌2가 지속적으로 진행되고 있는 만큼 지금 시기에 이에 대해 이야기...,1
...,...,...
164724,의 건립비로 지출하는 기부금 9 중소기업협동조합법 에 따른 중소기업중앙회에 중소기업...,6126
164725,이하 이 호에서 같다 가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경...,6126
164726,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,6126
164727,한국국제학교 22 천진한국국제학교 23 테헤란한국학교 ㅇ 법정기부금단체 중 전문모금...,6126


In [79]:
df2.to_csv("./산림복지/2019산림복지_본문_전처리.csv", index=False, encoding='UTF-8')

In [80]:
df2

,split_str,org_idx
0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,0
1,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤원해 평생학습원 이세인 ...,0
2,안녕하세요 메리썽입니다,1
3,혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는데요,1
4,현재 혁신도시 시즌2가 지속적으로 진행되고 있는 만큼 지금 시기에 이에 대해 이야기...,1
...,...,...
164724,의 건립비로 지출하는 기부금 9 중소기업협동조합법 에 따른 중소기업중앙회에 중소기업...,6126
164725,이하 이 호에서 같다 가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경...,6126
164726,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,6126
164727,한국국제학교 22 천진한국국제학교 23 테헤란한국학교 ㅇ 법정기부금단체 중 전문모금...,6126


## 맞춤법 검사

In [2]:
df2 = pd.read_csv("./산림복지/2019산림복지_본문_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx
0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,0
1,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤원해 평생학습원 이세인 ...,0
2,안녕하세요 메리썽입니다,1
3,혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는데요,1
4,현재 혁신도시 시즌2가 지속적으로 진행되고 있는 만큼 지금 시기에 이에 대해 이야기...,1
...,...,...
164724,의 건립비로 지출하는 기부금 9 중소기업협동조합법 에 따른 중소기업중앙회에 중소기업...,6126
164725,이하 이 호에서 같다 가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경...,6126
164726,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,6126
164727,한국국제학교 22 천진한국국제학교 23 테헤란한국학교 ㅇ 법정기부금단체 중 전문모금...,6126


In [3]:
#- 패키지 임포트
import requests
import json
import pandas as pd
from tqdm.notebook import tqdm
import re


'''맞춤법 검사 함수'''
def ko_grammar(sent):
    global correct_list ; correct_list= []
    #for i in tqdm(range(len(a))):
    for i in tqdm(range(len(sent))):
        try:
            print(i)                                                                               
            response = requests.post('http://164.125.7.61/speller/results', data={'text1': sent[i]})  # 맞춤법 검사 요청 (requests)    
            data = response.text.split('data = [', 1)[-1].rsplit('];', 1)[0] # 응답에서 필요한 내용 추출 (html 파싱)
            data = json.loads(data) # 파이썬 딕셔너리 형식으로 변환
            orgStr = [err['orgStr'] for err in data['errInfo']] #오류가 담긴 errinfo에서 고쳐야 되는 단어 리스트로 추출 
            correct = [err['candWord'] for err in data['errInfo']] #고쳐진 단어도 리스트로 추출
            splited = sent[i] 

            '''여러 개로 고쳐졌을 때 맨 처음 단어만 선택'''
            for j in range(len(correct)):
                if '|'in correct[j]: #여러 개로 고쳐질 경우 문자열이 '|'로 구분되어 있음
                    com = re.compile('\|')
                    many = com.search(correct[j])
                    correct[j]= correct [j][0:(many.span()[0])]
                '''틀린 문장 고치기'''   
                splited = splited.replace(orgStr[j], correct[j])

        except:      
            splited = sent[i]

        correct_list.append(splited)
        display.clear_output(wait=True) # 출력지우기

In [13]:
# 데이터프레임을 5000개 단위로 나누기
chunk_size = 5000
num_chunks = len(df2) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df2[start_idx:end_idx]
    chunk_df.reset_index(drop=True, inplace=True)
    dfs[f"df2_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df2_1
df2_2
df2_3
df2_4
df2_5
df2_6
df2_7
df2_8
df2_9
df2_10
df2_11
df2_12
df2_13
df2_14
df2_15
df2_16
df2_17
df2_18
df2_19
df2_20
df2_21
df2_22
df2_23
df2_24
df2_25
df2_26
df2_27
df2_28
df2_29
df2_30
df2_31
df2_32
df2_33


In [ ]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df_s = pd.DataFrame(columns = ['split_str', 'org_idx', 'correct_str']) # 새로운 데이터프레임 제작

# ''' 문장 데이터 프레임 저장 '''
# df_s.to_csv('./산림복지/2019산림복지_본문_맞춤법검사.csv', index=False, encoding='UTF-8')

In [2]:
df_s = pd.read_csv("./산림복지/2019산림복지_본문_맞춤법검사.csv", encoding='UTF-8')
df_s

,split_str,org_idx,correct_str
0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...
1,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤원해 평생학습원 이세인 ...,0,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤 원해 평생학습원 이세인...
2,안녕하세요 메리썽입니다,1,안녕하세요. 메리 성입니다
3,혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는데요,1,혁신도시에 대해서는 많이들 들어보셨을 거라고 생각이 드는데요
4,현재 혁신도시 시즌2가 지속적으로 진행되고 있는 만큼 지금 시기에 이에 대해 이야기...,1,현재 혁신도시 시즌2가 지속해 진행되고 있는 만큼 지금 시기에 이에 대해 이야기해보...
...,...,...,...
19995,이정도면 일주일 정도 먹어요 산양우유는 유통기한이 대략 일주일 정도라서 먹는 양을 ...,789,이 정도면 일주일 정도 먹어요. 산양 우유는 유통기한이 대략 일주일 정도라서 먹는 ...
19996,수암숲속목장 유기농 산양우유의 패키지는 민트와 핑크 이렇게 두 종류에요,789,유기농 산양 우유의 패키지는 박하와 핑크 이렇게 두 종류에요
19997,맛이 다른 건 아니구요 컬러는 랜덤으로 배송된답니다 ㅎㅎ,789,맛이 다른 건 아니고요 컬러는 무작위로 배송된답니다 하하
19998,패키지 컬러감이 예뻐서 소장각이에요 갖고 싶은 컬러는 주문할 때 써주면 최대한 반영...,789,패키지 색감이 예뻐서 소장 각이에요 갖고 싶은 컬러는 주문할 때 써주면 최대한 반영...


In [22]:
'''맞춤법 검사하기''' 
a = 4
while a < 38:
    ko_grammar(dfs["df2_%s"%a]['split_str'])
    dfs["df2_%s"%a]['correct_str'] = correct_list
    df_s = pd.concat([df_s,dfs["df2_%s"%a]])
    df_s.to_csv('./산림복지/2019산림복지_본문_맞춤법검사.csv', index=False, encoding='UTF-8')
    a += 1

830


In [11]:
df_c.to_csv('./산림복지/2019산림복지_본문_맞춤법검사.csv', encoding='UTF-8')